In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [ ]:
xgb = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/xgb_labels_semeval.csv')
new_deepmoji = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/new_deepmoji_semeval.csv')
new_bertweet = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/new_bertweet_semeval.csv')
ada = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/ada_labels_semeval.csv')
hist = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/hist_labels_semeval.csv')
rf = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/rf_labels_semeval.csv')
log = pd.read_csv('../Code/BMA/results_semeval/input/prediction_file/logi_labels_semeval.csv')

In [ ]:
def normalize_pred(x):
    if x == '2:0':
        i = 1
    else:
        i = 0
    return i

In [ ]:
def prepare_prediction(xgb):
    xgb.predicted = xgb.predicted.apply(normalize_pred)
    xgb.distribution = xgb.distribution.str.replace(r'*', '').apply(lambda x: np.float(x))
    xgb['Unnamed: 5'] = xgb['Unnamed: 5'].str.replace(r'*', '').apply(lambda x: np.float(x))
    xgb.actual = xgb.actual.apply(normalize_pred)
    xgb_distribution = xgb[['distribution', 'Unnamed: 5']]
    return xgb, xgb_distribution

In [ ]:
xgb, xgb_distribution = prepare_prediction(xgb)
new_deemoji, new_deepmoji_distribution = prepare_prediction(new_deepmoji)
new_bertweet, new_bertweet_distribution = prepare_prediction(new_bertweet)
ada, ada_distribution = prepare_prediction(ada)
hist, hist_distribution = prepare_prediction(hist)
rf, rf_distribution = prepare_prediction(rf)
log, log_distribution = prepare_prediction(log)

## Sum probability

In [ ]:
bma_0_lg = log_distribution.distribution * 0.676448
bma_0_xgb = xgb_distribution.distribution * 0.654673
bma_0_randomf = rf_distribution.distribution * 0.580815
bma_0_ada = ada_distribution.distribution * 0.629298
bma_0_hist = hist_distribution.distribution * 0.654013
bma_0 = bma_0_lg + bma_0_xgb + bma_0_ada + bma_0_hist

In [ ]:
distrbution_0 = pd.concat([bma_0, new_deepmoji_distribution.distribution, new_bertweet_distribution.distribution], axis = 1)

In [ ]:
dist_0_mean = distrbution_0.sum(axis = 1)

In [ ]:
bma_1_lg = log_distribution['Unnamed: 5']*0.672901
bma_1_xgb = xgb_distribution['Unnamed: 5'] *0.647892
bma_1_randomf = rf_distribution['Unnamed: 5'] * 0.578112
bma_1_ada = ada_distribution['Unnamed: 5'] * 0.634285
bma_1_hist = hist_distribution['Unnamed: 5'] * 0.643814
bma_1 = bma_1_lg + bma_1_xgb  + bma_1_hist + bma_1_ada

In [ ]:
distrbution_1 = pd.concat([bma_1, new_deepmoji_distribution['Unnamed: 5'], new_bertweet_distribution['Unnamed: 5']], axis = 1) # transformer_distribution['Unnamed: 5'], deepmoji_distribution['Unnamed: 5']]

In [ ]:
dist_1_mean = distrbution_1.sum(axis = 1)

In [ ]:
predict = pd.concat([dist_0_mean, dist_1_mean], axis = 1).values

In [ ]:
accuracy_score(xgb.actual, predict.argmax(axis = -1))

In [ ]:
print("Soft Classification Semeval")
print(classification_report(xgb.actual, predict.argmax(axis = -1)))

In [ ]:
f1_score(xgb.actual, predict.argmax(axis = -1))

In [ ]:
print(classification_report(xgb.actual, xgb.predicted))

In [ ]:
print(classification_report(xgb.actual, new_bertweet.predicted))

In [ ]:
print(classification_report(new_bertweet.actual, new_deepmoji.predicted))

## Majority Voting

In [ ]:
prdiction_bma = np.argmax(pd.concat([bma_0,bma_1], axis =1).values, -1)

In [ ]:
clf_list = pd.concat([pd.Series(prdiction_bma, name='predicted'),new_deemoji.predicted, new_bertweet.predicted], axis=1) #,transformer.predicted, deepmoji.predicted

In [ ]:
def majority_voting(clf_list):
    prediction = []
    for i, j in clf_list.iterrows():
        voting = Counter([j[0], j[1], j[2]])
        prediction.append(voting.most_common(1)[0][0])
    prediction = np.asarray(prediction)
    
    return prediction

In [ ]:
prediction = majority_voting(clf_list)

In [ ]:
print('Hard Classification Semeval')
print(classification_report(xgb.actual, prediction))

In [ ]:
f1_score(xgb.actual, prediction)